<a href="https://colab.research.google.com/github/purvi202211023/IT555-Deep-Neural-NLP/blob/main/TF_IDF_weighted_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---
**Team NeuronicNav**

Assignment 1: Train Word2Vec on peS2o Dataset (AllenNLP)

Id No.: 202211023, 202211004, 202221004

---




In [ ]:
import gzip
import json
import requests
import concurrent.futures
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

!pip install num2words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import inflect
import numpy as np
import random
from num2words import num2words  # For numerical-to-text conversion

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=f6108d7d8e6513e9b2d773ada5b26979fe03c0f9fcc1fee0c64bba4e4fa39b12
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset

In [ ]:
file_name = "/content/drive/MyDrive/NLP/dataset_pes2o.json.gz"
data = []

with gzip.open(file_name, 'rb') as f:
    for line in f:
        json_obj = json.loads(line)
        data.append(json_obj)

In [ ]:
df = pd.DataFrame(data)
df

,id,source,version,added,created,text
0,254721023,s2ag/valid,v1,2022-12-16T16:07:05.559Z,2022-12-14T00:00:00.000Z,Investigation Some Characteristics of Biocompo...
1,254200673,s2ag/valid,v1,2022-12-04T16:21:43.441Z,2022-12-02T00:00:00.000Z,Modelling Techniques for Conjunctive Water Man...
2,252643161,s2ag/valid,v1,2022-10-01T15:18:53.788Z,2022-12-01T00:00:00.000Z,Complete Targets Coverage in Energy Harvesting...
3,244852373,s2ag/valid,v1,2021-12-04T16:21:39.980Z,2022-12-01T00:00:00.000Z,Nonorthogonal Multiple Access Enabled Two-Way ...
4,254774363,s2ag/valid,v1,2022-12-17T16:19:10.635Z,2022-12-14T00:00:00.000Z,106 A CASE OF COMPLETE ATRIOVENTRICULAR BLOCK ...
...,...,...,...,...,...,...
111223,254393835,s2ag/valid,v1,2022-12-08T16:07:54.109Z,2022-12-05T00:00:00.000Z,On the Death of Diversity Jurisdiction: An Emp...
111224,254122749,s2ag/valid,v1,2022-12-02T06:17:23.245Z,2022-12-01T00:00:00.000Z,"""Electron Complementation""-Induced Molybdenum ..."
111225,254805941,s2ag/valid,v1,2022-12-18T16:13:29.904Z,2022-12-16T00:00:00.000Z,An Innovative Adaptation of an HIV Status-Neut...
111226,254613890,s2ag/valid,v1,2022-12-14T16:21:33.760Z,2022-12-12T00:00:00.000Z,DNA barcoding unveils a high diversity of cadd...


**Load .csv file of that dataset**

In [ ]:
data_df = pd.read_csv("/content/drive/MyDrive/NLP/peS2o Dataset.csv")

# Q1: Research papers are collected only from Computer science domain

In [ ]:
# Creating a dataframe that contains only the computer science papers as datarows
data_df = data_df[data_df["fieldsOfStudy"].str.lower().str.contains("computer science")]
data = data_df.iloc[:4000]
data

,paperId,id,title,fieldsOfStudy,source,version,added,created,text
0,c0b6a8dfe459a17b885902f1415c5f0acfd72cc5,252643161,Complete Targets Coverage in Energy Harvesting...,Computer Science,s2ag/valid,v1,2022-10-01T15:18:53.788Z,2022-12-01T00:00:00.000Z,Complete Targets Coverage in Energy Harvesting...
1,6602075f1e6fd8966e98de07be4230ec72b1ed0b,244852373,Nonorthogonal Multiple Access Enabled Two-Way ...,Computer Science,s2ag/valid,v1,2021-12-04T16:21:39.980Z,2022-12-01T00:00:00.000Z,Nonorthogonal Multiple Access Enabled Two-Way ...
2,4767015228fb30be57a21b626c7867239255b3b6,252033565,Event-Triggered Adaptive Output Feedback Contr...,Computer Science,s2ag/valid,v1,2022-09-03T15:09:09.625Z,2023-01-01T00:00:00.000Z,Event-Triggered Adaptive Output Feedback Contr...
3,f72832c96c65f630e5fedcc932442d38ca1be760,255290365,From MMU to MPU: adaptation of the Pip kernel ...,Computer Science,s2ag/valid,v1,2022-12-31T16:09:26.680Z,2022-12-22T00:00:00.000Z,From MMU to MPU: Adaptation of the Pip Kernel ...
4,ab61f83262b23dc81f77f6dc3e80cb81e2889b7d,249073215,Data-Driven Cyber-Attack Detection of Intellig...,Computer Science,s2ag/valid,v1,2022-05-27T15:22:24.852Z,2023-04-01T00:00:00.000Z,Data-Driven Cyber-Attack Detection of Intellig...
...,...,...,...,...,...,...,...,...,...
3995,4a1fd41ad3a3f87831e84880fb86bc49a16bbfde,250445263,Garbled-Circuits from an SCA Perspective: Free...,Computer Science,s2ag/valid,v1,2022-07-12T05:36:34.136Z,2023-03-06T00:00:00.000Z,Garbled-Circuits from an SCA Perspective\n\n. ...
3996,b59abd08f4fabb3637aca8cc4e91710765b12cf7,246769561,Low Parasitic-Inductance Packaging of a 650 V/...,Computer Science,s2ag/valid,v1,2022-02-12T16:24:40.593Z,2023-01-01T00:00:00.000Z,Low Parasitic-Inductance Packaging of a 650 V/...
3997,4a1ecae9038bc2736a8d03928170593cf6262ad2,232095868,Identifiability in the Behavioral Setting,Computer Science,s2ag/valid,v1,2021-03-02T17:10:31.474Z,2023-03-01T00:00:00.000Z,Identifiability in the Behavioral Setting\n\nT...
3998,4ca48db48f60e5eab29cc84890147c2c864c8a7e,253953469,Online and Real-Time Trajectory Generation Met...,Computer Science,s2ag/valid,v1,2022-11-26T16:30:16.963Z,2023-01-01T00:00:00.000Z,Online and Real-Time Trajectory Generation Met...


# Q2: 2. A random training data of 3000 documents are used, while a test dataset of 1000 documents is to be used.

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=1000, random_state=42)

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (3000, 9)
Test data shape: (1000, 9)


# Preprocessing

Q3: The linguistic unit (row vector) in your input context matrix should be sentences (and not words). That is, you are to embed sentences instead of words.

Q4: The following pre-processing cleanup needs to be done on the input context matrix: (i) stopwords, (ii) urls, (iii) bullets, (iv) apostrophe, (v) hyphens, (vi) enumerations (eg. like how this list is being enumerated), (vii), numerical-to-text conversion, (viii) punctuations.

In [ ]:
stop_words = stopwords.words("english")

def preprocess_text(text):
    # Removing stopwords using Regex

    # stopwords = set(["a", "an", "the", "in", "on", "at", "to", "of", "for", "with", "by"])
    # text = " ".join([word for word in text.split() if word.lower() not in stopwords])

    # Removing stopwords using NLTK
    text= " ".join([word for word in text.split() if word.lower() not in stop_words])

    # Removing URLs
    text = re.sub(r'http\S+', '', text)

    # Removing bullets
    text = re.sub(r'\s•\s', ' ', text)

    # Removing apostrophes
    text = re.sub(r"'s\b", '', text)

    # Removing hyphens
    text = re.sub(r'-', ' ', text)

    # Remove enumerations
    text = re.sub(r'\b\d+\.\s*', '', text)

    # Converting numerical values to text
    text = re.sub(r'\d+', lambda x: num2words(int(x.group())), text)

    # Removing punctuation
    # text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[^\w\s.]', '', text)
    return text.lower()  # Converting the text to lowercase

In [ ]:
data['text_processed'] = data['text'].apply(preprocess_text)

<ipython-input-25-da38a64fa62d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text_processed'] = data['text'].apply(preprocess_text)


In [ ]:
data['text_processed']

0       complete targets coverage energy harvesting in...
1       nonorthogonal multiple access enabled two way ...
2       event triggered adaptive output feedback contr...
3       mmu mpu adaptation pip kernel constrained devi...
4       data driven cyber attack detection intelligent...
                              ...                        
3995    garbled circuits sca perspective . garbling sc...
3996    low parasitic inductance packaging six hundred...
3997    identifiability behavioral setting behavioral ...
3998    online real time trajectory generation method ...
3999    investigation optimal pid like fuzzy logic con...
Name: text_processed, Length: 4000, dtype: object

In [ ]:
data.to_csv("/content/drive/MyDrive/NLP/preprocessed.csv")

In [ ]:
import numpy as np
import random
corpus = data["text_processed"].tolist()
corpus = ' '.join(corpus)
corpus = corpus.replace(".", "")
vocab = set(word for word in corpus.split(" "))
vocab_size = len(vocab)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

corpus = data["text_processed"].tolist()
corpus = ' '.join(corpus)
corpus=corpus.split(".")

# Hyperparameters
embedding_dim = 300
window_size = 3
learning_rate = 0.1
epochs = 3000

# Initialize sentence vectors randomly
sentence_vectors = (np.random.rand(len(corpus), embedding_dim) - 0.5)
# Sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Training the Word2Vec model with Skip-gram on sentences
for epoch in range(epochs):
    total_loss = 0

    for context_sentence_idx, context_sentence in enumerate(corpus):
        for center_sentence_idx in range(max(0, context_sentence_idx - window_size),
                                         min(len(corpus), context_sentence_idx + window_size + 1)):
            if context_sentence_idx != center_sentence_idx:
                context_sentence_vec = sentence_vectors[context_sentence_idx]
                center_sentence = corpus[center_sentence_idx]
                # Calculate the score
                score = np.dot(context_sentence_vec, sentence_vectors[center_sentence_idx])
                # Apply sigmoid to the score
                pred = sigmoid(score)

                # Calculate the loss
                loss = -np.log(pred)
                total_loss += loss

                # Compute the gradients and update sentence vectors
                grad = pred - 1
                sentence_vectors[context_sentence_idx] -= learning_rate * grad * sentence_vectors[center_sentence_idx]

    if epoch %50 == 0:
        print(f"Epoch {epoch + 1}, Loss: {total_loss}")
    if total_loss <= 0.1:
        print(f"Epoch {epoch + 1}, Loss: {total_loss}")
        break

Epoch 1, Loss: 117150.96662694495
Epoch 51, Loss: 447.98597095804485
Epoch 101, Loss: 210.90088510692922
Epoch 151, Loss: 135.60050785339737
Epoch 201, Loss: 99.11535035203013
Epoch 251, Loss: 77.72969726833661
Epoch 301, Loss: 63.73501835009047
Epoch 351, Loss: 53.891639363829164
Epoch 401, Loss: 46.605520620258226
Epoch 451, Loss: 41.00307986446487
Epoch 501, Loss: 36.566347405723434
Epoch 551, Loss: 32.96914673289042
Epoch 601, Loss: 29.996062882095732
Epoch 651, Loss: 27.499228168343368
Epoch 701, Loss: 25.373842380969403
Epoch 751, Loss: 23.543601153827627
Epoch 801, Loss: 21.95165501729455
Epoch 851, Loss: 20.554798819965118
Epoch 901, Loss: 19.319622328461893
Epoch 951, Loss: 18.219891893630255
Epoch 1001, Loss: 17.234727725601765
Epoch 1051, Loss: 16.347308715985196
Epoch 1101, Loss: 15.543935121125068
Epoch 1151, Loss: 14.813338992691943
Epoch 1201, Loss: 14.14616929309583
Epoch 1251, Loss: 13.534602236217207
Epoch 1301, Loss: 12.9720427587285
Epoch 1351, Loss: 12.452893227428

# TF-IDF weighted word2vec

In [ ]:
# Code using class and functions

class Word2Vec:
    def __init__(self, corpus, embedding_dim=300, window_size=3, learning_rate=0.1, epochs=2000):
        self.corpus = corpus
        self.embedding_dim = embedding_dim
        self.window_size = window_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.vocab = set(word for sentence in corpus for word in sentence)
        self.vocab_size = len(self.vocab)
        self.word_to_idx = {word: idx for idx, word in enumerate(self.vocab)}
        self.idx_to_word = {idx: word for word, idx in self.word_to_idx.items()}
        self.sentence_vectors = (np.random.rand(len(corpus), embedding_dim) - 0.5)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def train(self):
        for epoch in range(self.epochs):
            total_loss = 0

            for context_sentence_idx, context_sentence in enumerate(self.corpus):
                for center_sentence_idx in range(max(0, context_sentence_idx - self.window_size),
                                                 min(len(self.corpus), context_sentence_idx + self.window_size + 1)):
                    if context_sentence_idx != center_sentence_idx:
                        context_sentence_vec = self.sentence_vectors[context_sentence_idx]
                        center_sentence = self.corpus[center_sentence_idx]

                        # Calculate the score
                        score = np.dot(context_sentence_vec, self.sentence_vectors[center_sentence_idx])

                        # Apply sigmoid to the score
                        pred = self.sigmoid(score)

                        # Calculate the loss
                        loss = -np.log(pred)
                        total_loss += loss

                        # Compute the gradients and update sentence vectors
                        grad = pred - 1
                        self.sentence_vectors[context_sentence_idx] -= self.learning_rate * grad * self.sentence_vectors[center_sentence_idx]

            if True:
                print(f"Epoch {epoch + 1}, Loss: {total_loss}")
            if total_loss <= 0.1:
                print(f"Epoch {epoch + 1}, Loss: {total_loss}")
                break

    def sentence_to_vector(self, sentence):
        words = sentence.split()
        vector = np.zeros(self.embedding_dim)
        for word in words:
            if word in self.word_to_idx:
                vector += self.sentence_vectors[self.word_to_idx[word]]
        norm = np.linalg.norm(vector)
        if norm != 0:
            vector /= norm
        return vector

    def predict_similar_sentences(self, new_sentence, num_similar=5):
        default_vector = np.zeros(self.embedding_dim)
        new_sentence = new_sentence.lower()
        new_sentence_vector = self.sentence_to_vector(new_sentence)

        similarities = np.dot(self.sentence_vectors, new_sentence_vector)
        similar_sentence_indices = np.argsort(similarities)[::-1][:num_similar]
        similar_sentences = [self.corpus[idx] for idx in similar_sentence_indices]

        return similar_sentences

if __name__ == "__main__":
    corpus = data["text_processed"].tolist()
    corpus = ' '.join(corpus)


    word2vec_model = Word2Vec(corpus)
    word2vec_model.train()

In [ ]:
def calculate_tfidf_in_context(sentences, window_size=5):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([" ".join(sentence) for sentence in sentences])

    context_tfidf = {}

    for sentence in sentences:
        for target_index, target_word in enumerate(sentence):
            context_start = max(0, target_index - window_size)
            context_end = min(len(sentence), target_index + window_size + 1)
            context_words = sentence[context_start:target_index] + sentence[target_index+1:context_end]

            context_tfidf[target_word] = {}

            for word in context_words:
                word_index = tfidf_vectorizer.vocabulary_.get(word, -1)
                if word_index != -1:
                    context_tfidf[target_word][word] = tfidf_matrix[target_index, word_index]

    return context_tfidf

In [ ]:
corpus = data["text_processed"].tolist()
corpus = ' '.join(corpus)
corpus=corpus.split(".")
corpus

['complete targets coverage energy harvesting internet things ambient backscatter article considers deriving set covers set active nodes responsible monitoring targets internet things iot network',
 ' key distinction prior works article considers sensor nodes aided backscatter communications allow communicate negligible energy cost using ambient radio frequency rf signals',
 ' article contains three main novelties',
 ' first presents mixed integer linear program milp used compute global optimal solution',
 ' second also outlines centralized greedy scheduling cgs algorithm selects nodes based energy number covered targets',
 ' third presents distributed greedy scheduling dgs algorithm selects nodes according energy level',
 ' simulation results show equipping nodes ambient backscattering capability increases ratio complete targets coverage one hundred compared existing techniques',
 ' nonorthogonal multiple access enabled two way relay system using signal alignment conventional two way 

In [ ]:
# Calculating TF-IDF for the corpus
Mc = calculate_tfidf_in_context(corpus, window_size=5)
Wt = (np.random.rand(Mc.shape[1], embedding_dim) - 0.5)  # Wt
Wc = (np.random.rand(Mc.shape[1], embedding_dim) - 0.5)  # Wc

In [ ]:
epochs = 100
window_size = 2
learning_rate = 0.01

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Binary Cross-Entropy Loss Function
def binary_cross_entropy_loss(y_true, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Number of sentences to use in each batch
batch_size = 100

# Training the Word2Vec model with Skip-gram on sentences
for epoch in range(epochs):
    total_loss = 0
    shuffled_indices = np.random.permutation(len(corpus))

    for batch_start in range(0, len(corpus), batch_size):
        batch_indices = shuffled_indices[batch_start:batch_start + batch_size]

        for context_sentence_idx in batch_indices:
            context_vector = Mc[context_sentence_idx]
            z = np.dot(Wc, np.dot(Wt.T, context_vector))
            y_pred = sigmoid(z)
            y_true = context_vector
            loss = binary_cross_entropy_loss(y_true, y_pred)

            # Update Wt and Wc using gradient descent

            gradient_Wc = np.dot((y_pred - y_true).reshape(-1, 1), np.dot(Wt.T, context_vector).reshape(-1, 1).T)
            gradient_Wt = np.dot((y_pred - y_true).reshape(-1, 1), np.dot(Wc.T, context_vector).reshape(-1, 1).T)

            Wt -= learning_rate * gradient_Wt
            Wc -= learning_rate * gradient_Wc

            total_loss += loss

    if epoch % 1 == 0:
        print(f"Epoch {epoch + 1}, Loss: {total_loss}")

    if total_loss <= 0.1:
        print(f"Epoch {epoch + 1}, Loss: {total_loss}")
        break

In [ ]:
def sentence_to_vector(sentence, word_to_idx, sentence_vectors, default_vector=None):
    words = sentence.split()
    vector = np.zeros(sentence_vectors.shape[1])
    for word in words:
        if word in word_to_idx:
          vector += sentence_vectors[word_to_idx[word]]


    # Normalize the vector (optional but recommended)
    norm = np.linalg.norm(vector)
    if norm != 0:
        vector /= norm
    return vector

In [ ]:
new_sentence_1 = "A Practical Way for Parasitics and Noise Tolerance in Analog IMC Circuits"
new_sentence_1=preprocess_text(new_sentence_1)
new_sentence_2 = "Fully analog IMC circuits remove the need for signal converters by realizing both MVM and NLV operations in the analog domain leading to significant energy savings"
new_sentence_2=preprocess_text(new_sentence_2)
vector_1 = sentence_to_vector(new_sentence_1, word_to_idx, sentence_vectors)
vector_2 = sentence_to_vector(new_sentence_2, word_to_idx, sentence_vectors)

# Calculate cosine similarity between sentence and existing sentences
similarities_1 = np.dot(sentence_vectors, vector_1)
similarities_2 = np.dot(sentence_vectors, vector_2)

# Find the most similar sentence for each new sentence
most_similar_sentence_1 = corpus[np.argmax(similarities_1)]
most_similar_sentence_2 = corpus[np.argmax(similarities_2)]

print(f"Most similar sentence to : '{most_similar_sentence_1}'")
print(f"Most similar sentence to : '{most_similar_sentence_2}'")

# Find the most similar sentence for each new sentence
most_disimilar_sentence_1 = corpus[np.argmin(similarities_1)]
most_disimilar_sentence_2 = corpus[np.argmin(similarities_2)]

print(f"Most disimilar sentence to : '{most_disimilar_sentence_1}'")
print(f"Most disimilar sentence to : '{most_disimilar_sentence_2}'")

Most similar sentence to : ' filtering uninformative variables also important knowledge discovery'
Most similar sentence to : ' proposed sic jfet vco serve fundamental building block wireless sensing data communication extreme temperature industrial space applications'
Most disimilar sentence to : ' comparative study outlier detection methods high dimensional text data abstract outlier detection aims find data sample significantly different data samples'
Most disimilar sentence to : ' blockchain based model tracking compliance security requirements increasing threat landscape industrial control systems ics brings different risk profiles comprehensive impacts society safety'
